In [9]:
import pandas as pd
import numpy as np
import random
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
import scipy.optimize
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

In [2]:
# Tạo danh sách người dùng và đánh giá
users = []
reviews = []

for i in range(1, 100):
    user_name = i
    for j in range(1, 40):
        product = random.randint(1, 300)
        rating = random.randint(1, 5)
        reviews.append([user_name, product, rating])

# Tạo DataFrame
df = pd.DataFrame(reviews, columns=['user_name', 'product_id', 'rating'])


In [3]:
df.head()

,user_name,product_id,rating
0,1,74,1
1,1,95,5
2,1,207,4
3,1,191,1
4,1,115,3


In [14]:
def Myrecommend():
    def normalizeRatings(myY, myR):
        # Tính trung bình các đánh giá
        Ymean = np.sum(myY, axis=1) / np.sum(myR, axis=1)
        Ymean = Ymean.reshape((Ymean.shape[0], -1))
        return myY - Ymean, Ymean

    def flattenParams(myX, myTheta):
        return np.concatenate((myX.flatten(), myTheta.flatten()))

    def reshapeParams(flattened_XandTheta, mynm, mynu, mynf):
        assert flattened_XandTheta.shape[0] == int(mynm * mynf + mynu * mynf)
        reX = flattened_XandTheta[: int(mynm * mynf)].reshape((mynm, mynf))
        reTheta = flattened_XandTheta[int(mynm * mynf) :].reshape((mynu, mynf))
        return reX, reTheta

    def cofiCostFunc(myparams, myY, myR, mynu, mynm, mynf, mylambda=0.0):
        myX, myTheta = reshapeParams(myparams, mynm, mynu, mynf)
        term1 = myX.dot(myTheta.T)
        term1 = np.multiply(term1, myR)
        cost = 0.5 * np.sum(np.square(term1 - myY))
        # Thêm phần regularization
        cost += (mylambda / 2.0) * np.sum(np.square(myTheta))
        cost += (mylambda / 2.0) * np.sum(np.square(myX))
        return cost

    def cofiGrad(myparams, myY, myR, mynu, mynm, mynf, mylambda=0.0):
        myX, myTheta = reshapeParams(myparams, mynm, mynu, mynf)
        term1 = myX.dot(myTheta.T)
        term1 = np.multiply(term1, myR)
        term1 -= myY
        Xgrad = term1.dot(myTheta)
        Thetagrad = term1.T.dot(myX)
        # Thêm phần regularization
        Xgrad += mylambda * myX
        Thetagrad += mylambda * myTheta
        return flattenParams(Xgrad, Thetagrad)

    
    mynu = df_train.user_name.nunique()
    mynm = df_train.product_id.nunique()
    mynf = 10
    Y = np.zeros((mynm, mynu))

    user_to_column = {user_name: idx for idx, user_name in enumerate(df['user_name'].unique())}
    product_to_row = {product: idx for idx, product in enumerate(df['product_id'].unique())}

    for row in df.itertuples():
        Y[product_to_row[row.product_id], user_to_column[row.user_name]] = row.rating

    R = (Y != 0).astype(int)

    Ynorm, Ymean = normalizeRatings(Y, R)
    X = np.random.rand(mynm, mynf)
    Theta = np.random.rand(mynu, mynf)
    myflat = flattenParams(X, Theta)

    result = scipy.optimize.minimize(
        fun=cofiCostFunc,
        x0=myflat,
        args=(Ynorm, R, mynu, mynm, mynf, 3),
        method="TNC",
        jac=cofiGrad,
        options={"maxiter": 300},
    )

    resX, resTheta = reshapeParams(result.x, mynm, mynu, mynf)
    prediction_matrix = resX.dot(resTheta.T)
    return prediction_matrix, Ymean, product_to_row, user_to_column

In [5]:
class uuCF(object):
    def __init__(self, Y_data, k, sim_func=cosine_similarity):
        self.Y_data = np.array(Y_data)
        self.k = k
        self.sim_func = sim_func
        self.Ybar = None
        self.n_users = int(np.max(self.Y_data[:, 0])) + 1
        self.n_items = int(np.max(self.Y_data[:, 1])) + 1

    def fit(self):
        users = self.Y_data[:, 0]
        self.Ybar = self.Y_data.copy()
        self.mu = np.zeros((self.n_users,))
        for n in range(self.n_users):
            ids = np.where(users == n)[0].astype(np.int32)
            ratings = self.Y_data[ids, 2]
            self.mu[n] = np.mean(ratings) if ids.size > 0 else 0
            self.Ybar[ids, 2] = ratings - self.mu[n]
        self.Ybar = sparse.coo_matrix(
            (self.Ybar[:, 2], (self.Ybar[:, 1], self.Ybar[:, 0])), (self.n_items, self.n_users)
        ).tocsr()
        self.S = self.sim_func(self.Ybar.T, self.Ybar.T)

    def pred(self, u, i):
        ids = np.where(self.Y_data[:, 1] == i)[0].astype(np.int32)
        users_rated_i = (self.Y_data[ids, 0]).astype(np.int32)
        sim = self.S[u, users_rated_i]
        nns = np.argsort(sim)[-self.k :]
        nearest_s = sim[nns]
        r = self.Ybar[i, users_rated_i[nns]]
        eps = 1e-8
        return (r * nearest_s).sum() / (np.abs(nearest_s).sum() + eps) + self.mu[u]

In [6]:
# Tạo data frame với thông tin người dùng, sản phẩm và đánh giá
Y_data = df[['user_name', 'product_id', 'rating']].values

# Khởi tạo uuCF và chạy thử nghiệm
model = uuCF(Y_data, 5)
model.fit()

# Ví dụ dự đoán
user_id = 1  # Giả sử user_id là 1
item_id = 10  # Giả sử item_id là 10
print(f"Dự đoán đánh giá của user {user_id} cho sản phẩm {item_id} là: {model.pred(user_id, item_id)}")

Dự đoán đánh giá của user 1 cho sản phẩm 10 là: 3.220078164399729


In [15]:

# Tạo data frame với thông tin người dùng, sản phẩm và đánh giá
Y_data = df[['user_name', 'product_id', 'rating']].values

# Khởi tạo và chạy thử nghiệm cho Myrecommend
prediction_matrix, Ymean, product_to_row, user_to_column = Myrecommend()

# Tạo Y_pred cho Myrecommend
Y_pred = np.zeros_like(Y_data[:, 2])
for idx, (user, product_id, rating) in enumerate(Y_data):
    Y_pred[idx] = prediction_matrix[product_to_row[product_id], user_to_column[user]] + Ymean[product_to_row[product_id]]

# Khởi tạo uuCF và chạy thử nghiệm
model = uuCF(Y_data, 5)
model.fit()

# Tạo Y_pred cho uuCF
Y_pred_uuCF = np.zeros_like(Y_data[:, 2])
for idx, (user, product_id, rating) in enumerate(Y_data):
    Y_pred_uuCF[idx] = model.pred(user, product)

# Tính sai số cho cả hai phương pháp
mae_myrecommend = mean_absolute_error(Y_data[:, 2], Y_pred)
rmse_myrecommend = mean_squared_error(Y_data[:, 2], Y_pred, squared=False)

mae_uucf = mean_absolute_error(Y_data[:, 2], Y_pred_uuCF)
rmse_uucf = mean_squared_error(Y_data[:, 2], Y_pred_uuCF, squared=False)

print(f"MAE của Myrecommend: {mae_myrecommend}")
print(f"RMSE của Myrecommend: {rmse_myrecommend}")

print(f"MAE của uuCF: {mae_uucf}")
print(f"RMSE của uuCF: {rmse_uucf}")

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_13592\1027801243.py:58: OptimizeWarning: Unknown solver options: maxiter
  result = scipy.optimize.minimize(
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_13592\3292246476.py:10: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  Y_pred[idx] = prediction_matrix[product_to_row[product_id], user_to_column[user]] + Ymean[product_to_row[product_id]]


MAE của Myrecommend: 1.2364672364672364
RMSE của Myrecommend: 1.5192795359019051
MAE của uuCF: 1.2662522662522662
RMSE của uuCF: 1.5495778138549225


In [16]:
# Tạo data frame với thông tin người dùng, sản phẩm và đánh giá
Y_data = df[['user_name', 'product_id', 'rating']].values

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
train_data, test_data = train_test_split(Y_data, test_size=0.2, random_state=42)

# Tạo DataFrame cho tập huấn luyện
df_train = pd.DataFrame(train_data, columns=['user_name', 'product_id', 'rating'])

# Khởi tạo và chạy thử nghiệm cho Myrecommend với tham số tối ưu
prediction_matrix, Ymean, product_to_row, user_to_column = Myrecommend()

# Tạo Y_pred cho Myrecommend trên tập kiểm tra
Y_pred_test = np.zeros_like(test_data[:, 2])
for idx, (user, product_id, rating) in enumerate(test_data):
    if product_id in product_to_row and user in user_to_column:
        Y_pred_test[idx] = prediction_matrix[product_to_row[product_id], user_to_column[user]] + Ymean[product_to_row[product_id]]
    else:
        Y_pred_test[idx] = np.mean(df_train['rating'])  # Giá trị trung bình nếu sản phẩm hoặc người dùng mới

# Khởi tạo uuCF và chạy thử nghiệm với k tối ưu
model = uuCF(train_data, k=10)
model.fit()

# Tạo Y_pred cho uuCF trên tập kiểm tra
Y_pred_uuCF_test = np.zeros_like(test_data[:, 2])
for idx, (user, product_id, rating) in enumerate(test_data):
    Y_pred_uuCF_test[idx] = model.pred(user, product_id)

# Tính sai số cho cả hai phương pháp trên tập kiểm tra
mae_myrecommend_test = mean_absolute_error(test_data[:, 2], Y_pred_test)
rmse_myrecommend_test = mean_squared_error(test_data[:, 2], Y_pred_test, squared=False)

mae_uucf_test = mean_absolute_error(test_data[:, 2], Y_pred_uuCF_test)
rmse_uucf_test = mean_squared_error(test_data[:, 2], Y_pred_uuCF_test, squared=False)

print(f"MAE của Myrecommend trên tập kiểm tra: {mae_myrecommend_test}")
print(f"RMSE của Myrecommend trên tập kiểm tra: {rmse_myrecommend_test}")

print(f"MAE của uuCF trên tập kiểm tra: {mae_uucf_test}")
print(f"RMSE của uuCF trên tập kiểm tra: {rmse_uucf_test}")

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_13592\1027801243.py:58: OptimizeWarning: Unknown solver options: maxiter
  result = scipy.optimize.minimize(
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_13592\2672067891.py:17: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  Y_pred_test[idx] = prediction_matrix[product_to_row[product_id], user_to_column[user]] + Ymean[product_to_row[product_id]]


MAE của Myrecommend trên tập kiểm tra: 1.2703751617076326
RMSE của Myrecommend trên tập kiểm tra: 1.5453473131437543
MAE của uuCF trên tập kiểm tra: 1.4566623544631307
RMSE của uuCF trên tập kiểm tra: 1.739131840935367
